In [1]:
from sklearn.utils.multiclass import type_of_target

In [2]:
type_of_target([0,1])

'binary'

In [3]:
type_of_target([1,-1])

'binary'

In [4]:
type_of_target([0,1,2])

'multiclass'

In [5]:
type_of_target([1, 2.1, 3.4])

'continuous'

In [6]:
type_of_target([[-1,1],[1,-1]])

'multilabel-indicator'

In [7]:
type_of_target([[0,1],[1,1],[1,0]])

'multilabel-indicator'

In [8]:
type_of_target([[0,1,2],[0,1,2]])

'multiclass-multioutput'

In [9]:
type_of_target([[0,1],[0,2]])

'multiclass-multioutput'

In [10]:
type_of_target([[-1.2,1],[1,-1.2]])

'continuous-multioutput'

### Predict digit from image

In [1]:
import numpy as np
import os
import io
import warnings

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import Perceptron
from sklearn.metrics import hinge_loss, ConfusionMatrixDisplay, confusion_matrix, make_scorer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, classification_report
from sklearn.model_selection import cross_validate, cross_val_predict, GridSearchCV
from pprint import pprint


To start with, we will treat this as a binary classification problem.

In [2]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)